In [1]:
!pip install -U keras-nlp
!pip install -U keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.17.1 which is incompatible.


In [2]:
import keras
import keras_nlp
import numpy as np

In [3]:
import os

os.environ["KERAS_BACKEND"] = "jax"
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [4]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

In [5]:
import csv

data = []
template = "Question:\n{question}\n\nAnswer:\n{answer}"

with open("/kaggle/input/layoutlm/medquad.csv", mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        data.append(template.format(**row))

In [6]:
data[1]

'Question:\nWhat causes Glaucoma ?\n\nAnswer:\nNearly 2.7 million people have glaucoma, a leading cause of blindness in the United States. Although anyone can get glaucoma, some people are at higher risk. They include - African-Americans over age 40  - everyone over age 60, especially Hispanics/Latinos  - people with a family history of glaucoma. African-Americans over age 40 everyone over age 60, especially Hispanics/Latinos people with a family history of glaucoma.  In addition to age, eye pressure is a risk factor. Whether you develop glaucoma depends on the level of pressure your optic nerve can tolerate without being damaged. This level is different for each person. Thats why a comprehensive dilated eye exam is very important. It can help your eye care professional determine what level of eye pressure is normal for you. Another risk factor for optic nerve damage relates to blood pressure. Thus, it is important to also make sure that your blood pressure is at a proper level for you

In [7]:
prompt = template.format(
    question="What is (are) Diabetes ?",
    answer="",
)
print(gemma_lm.generate(prompt, max_length=256))

Question:
What is (are) Diabetes ?

Answer:
Diabetes is a group of diseases that affect the way your body uses blood sugar (glucose). Glucose is a type of sugar that is found in the foods you eat. It is also made by your liver and pancreas.

Diabetes is a chronic disease that affects the way your body uses blood sugar. It is caused by a problem with the way your body makes or uses insulin. Insulin is a hormone that helps glucose enter your cells to be used for energy.

There are two main types of diabetes:

* Type 1 diabetes: This is when your body doesn't make enough insulin. It is also called insulin-dependent diabetes.
* Type 2 diabetes: This is when your body doesn't use insulin well. It is also called non-insulin-dependent diabetes.

Diabetes can be managed with diet, exercise, and medication.

Diabetes is a chronic disease that affects the way your body uses blood sugar. It is caused by a problem with the way your body makes or uses insulin. Insulin is a hormone that helps glucos

In [8]:
prompt = template.format(
    question="How to prevent Diabetes ?",
    answer="",
)
print(gemma_lm.generate(prompt, max_length=256))

Question:
How to prevent Diabetes ?

Answer:
Diabetes is a disease that affects the way your body uses sugar (glucose). It is caused by a lack of insulin, a hormone that helps glucose enter your cells.

The pancreas is a gland that produces insulin. When you eat, your body releases insulin to help glucose enter your cells. If you have diabetes, your body either doesn’t make enough insulin or doesn’t use it properly.

Diabetes is a chronic disease that can lead to serious health problems, including heart disease, stroke, kidney disease, and blindness.

There is no cure for diabetes, but there are many ways to manage it.

The best way to prevent diabetes is to eat a healthy diet, exercise regularly, and maintain a healthy weight.

If you have diabetes, it’s important to monitor your blood sugar levels and take your medications as prescribed.

If you have diabetes, it’s important to monitor your blood sugar levels and take your medications as prescribed.

If you have diabetes, it’s import

In [9]:
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.preprocessor.sequence_length = 1281221
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

16412/16412 ━━━━━━━━━━━━━━━━━━━━ 6735s 407ms/step - loss: 1.3138 - sparse_categorical_accuracy: 0.6691


In [10]:
prompt = template.format(
    question="What is (are) Diabetes ?",
    answer="",
)
print(gemma_lm.generate(prompt, max_length=256))

Question:
What is (are) Diabetes ?

Answer:
Diabetes is a disease in which the body does not produce enough insulin or cannot use the insulin it produces properly. Insulin is a hormone that helps move sugar, or glucose, from the blood into the cells of the body. Glucose is the body's main source of energy.
                
Diabetes is a chronic disease that can be managed with diet, exercise, and medication. People with diabetes need to monitor their blood glucose levels and adjust their diet and exercise to keep their blood glucose levels within a normal range.
                
Diabetes is a major cause of blindness, kidney failure, and lower


In [11]:
prompt = template.format(
    question="How to prevent Diabetes ?",
    answer="",
)
print(gemma_lm.generate(prompt, max_length=256))

Question:
How to prevent Diabetes ?

Answer:
The following steps can help prevent or delay the onset of type 2 diabetes:
                
- Eat a healthy diet.  - Exercise regularly.  - Maintain a healthy weight.  - Avoid or limit alcohol.  - Avoid or limit smoking.  - Avoid or limit sugary drinks.  - Avoid or limit processed foods.  - Avoid or limit foods high in fat and sugar.  - Avoid or limit foods high in salt.  - Avoid or limit foods high in saturated fat.  - Avoid or limit foods high in trans fat.


In [16]:
gemma_lm.save("version_finetuned.keras")